In [ ]:
!pip install  git+https://github.com/TouiNishikawa/scCMGAN.git

In [2]:
import pandas as pd
import time
import os
from scCMGAN import scCMGAN
import warnings
warnings.simplefilter('ignore', FutureWarning)

In [ ]:
!git clone https://github.com/TouiNishikawa/scCMGAN.git

In [ ]:
%cd "./scCMGAN/example"

load data

In [8]:
# Path to table data
df = pd.read_csv('./Example_train.csv', header=None) 

# Name of dataset
data_set = "Example"

df.iloc[0, 0] = "cellType"
df = df.T
df.columns = df.iloc[0]
df = df.drop(df.index[[0]])

cellType = ["cell_1",
            "cell_2",
            "cell_3"]

## Short usage

In [9]:
# generate model
model = scCMGAN(persent = 40)

# finetuning model
model.fit(df, "cell_1", epochs=40)

# save model
model.save("saved_model.pkl")

# load model
model = scCMGAN.load("saved_model.pkl")

# generate data
synthetic_data = model.sample(10)

In [10]:
synthetic_data

,cellType,gene_1,gene_3,gene_2,gene_9,gene_4,gene_5,gene_6,gene_7,gene_8,gene_10
0,cell_1,1.968591,-0.001169,0.000621,-0.001316,0.0,0.0,0.0,0.0,0.0,0.0
1,cell_1,2.783189,0.001371,-0.000228,-0.000756,0.0,0.0,0.0,0.0,0.0,0.0
2,cell_1,4.084688,0.000234,-0.00101,-0.000119,0.0,0.0,0.0,0.0,0.0,0.0
3,cell_1,2.765709,0.001911,-0.002635,-0.001022,0.0,0.0,0.0,0.0,0.0,0.0
4,cell_1,2.144245,-0.000476,-0.000688,0.00107,0.0,0.0,0.0,0.0,0.0,0.0
5,cell_1,1.136047,-0.000766,-0.000482,-0.001979,0.0,0.0,0.0,0.0,0.0,0.0
6,cell_1,0.078797,-0.000106,0.002359,-0.001695,0.0,0.0,0.0,0.0,0.0,0.0
7,cell_1,1.171187,-0.001992,0.001992,-0.002129,0.0,0.0,0.0,0.0,0.0,0.0
8,cell_1,1.539754,-0.001405,0.001623,0.000785,0.0,0.0,0.0,0.0,0.0,0.0
9,cell_1,2.409395,0.002179,0.00164,-0.001655,0.0,0.0,0.0,0.0,0.0,0.0


## Training and generating

### N = 1

In [11]:
# persentage of cell marker genes
persent = 40 

# epoch: [epoch_start, epoch_end+1, epoch_term]
epoch_start = 50
epoch_end = 300
epoch_term = 50

for i in range(len(cellType)):
    # pick up
    ct =  cellType[i]
    df_celtype = df[df["cellType"]==ct]
    print("number of '" + ct + "': " + str(len(df_celtype)))
    for epoch in range(epoch_start, epoch_end+1, epoch_term ):
        if epoch == epoch_start:
            model = scCMGAN(persent)
        else:
            model = scCMGAN.load(data_set +'_scCMGAN'+str(persent)+'_' + ct + '_'+str(epoch-epoch_term)+'.pkl')
        
        # training
        print("epoch " + str(epoch)+" ")
        t1 = time.time() 
        model.fit(df, ct, 50)
        t2 = time.time()
        elapsed_time = t2-t1
        print(str(round(elapsed_time, 2)) + " seconds")
        
        model.save(data_set +'_scCMGAN'+str(persent)+'_' + ct + '_'+str(epoch)+'.pkl')

number of 'cell_1': 5
epoch 50 
1.96 seconds
epoch 100 
1.93 seconds
epoch 150 
2.0 seconds
epoch 200 
2.54 seconds
epoch 250 
3.01 seconds
epoch 300 
2.01 seconds
number of 'cell_2': 5
epoch 50 
2.03 seconds
epoch 100 
1.97 seconds
epoch 150 
1.96 seconds
epoch 200 
1.92 seconds
epoch 250 
2.96 seconds
epoch 300 
2.67 seconds
number of 'cell_3': 5
epoch 50 
1.97 seconds
epoch 100 
1.98 seconds
epoch 150 
1.96 seconds
epoch 200 
1.96 seconds
epoch 250 
2.16 seconds
epoch 300 
2.97 seconds


In [12]:
# num_data: [num_data_start, num_data_end+1, num_data_term]
num_data_start = 50
num_data_end = 500
num_data_term = 50

for i in range(len(cellType)):
    ct =  cellType[i]
    df_celtype = df[df["cellType"]==ct]

    for epoch in range(epoch_start, epoch_end+1, epoch_term):
        model = scCMGAN.load(data_set +'_scCMGAN'+str(persent)+'_' + ct + '_'+str(epoch)+'.pkl')

        for num_data in range(num_data_start, num_data_end+1, num_data_term):
                    t1 = time.time() 
                    synthetic_data = model.sample(num_data)
                    t2 = time.time()
                    elapsed_time = t2-t1
                    print(ct +"_"+ str(epoch) +"_"+str(num_data) + ":" + str(elapsed_time))
                    synthetic_data.to_csv("./synthetic_data_"+data_set+"_scCMGAN"+str(persent)+"_"+ ct +"_"+str(epoch)+"_" +  str(num_data) + ".csv", index= False)

cell_1_50_50:0.08530449867248535
cell_1_50_100:0.13785409927368164
cell_1_50_150:0.1893303394317627
cell_1_50_200:0.23393464088439941
cell_1_50_250:0.29352903366088867
cell_1_50_300:0.26897287368774414
cell_1_50_350:0.2691175937652588
cell_1_50_400:0.27696776390075684
cell_1_50_450:0.30519795417785645
cell_1_50_500:0.3726992607116699
cell_1_100_50:0.05266404151916504
cell_1_100_100:0.09174489974975586
cell_1_100_150:0.13912343978881836
cell_1_100_200:0.1696326732635498
cell_1_100_250:0.20764970779418945
cell_1_100_300:0.34787511825561523
cell_1_100_350:0.27072668075561523
cell_1_100_400:0.4372563362121582
cell_1_100_450:0.32182812690734863
cell_1_100_500:0.46901917457580566
cell_1_150_50:0.04990386962890625
cell_1_150_100:0.09155035018920898
cell_1_150_150:0.13042831420898438
cell_1_150_200:0.15736007690429688
cell_1_150_250:0.20219755172729492
cell_1_150_300:0.21517705917358398
cell_1_150_350:0.26384711265563965
cell_1_150_400:0.28554439544677734
cell_1_150_450:0.3068253993988037
cell

### N = 2 

In [13]:
# persentage of cell marker genes
persent = 20

# epoch: [epoch_start, epoch_end+1, epoch_term]
epoch_start = 50
epoch_end = 300
epoch_term = 50

for i in range(len(cellType)):
    # pick up
    ct =  cellType[i]
    df_celtype = df[df["cellType"]==ct]

    for epoch in range(epoch_start, epoch_end+1, epoch_term ):
        if epoch == epoch_start:
            model = scCMGAN(persent)
        else:
            model = scCMGAN.load(data_set +'_scCMGAN'+str(persent)+'_' + ct + '_'+str(epoch-epoch_term)+'.pkl')
        
        # training
        print("number of '" + ct + "': " + str(len(df_celtype)))
        print("epoch " + str(epoch)+" ")
        t1 = time.time() 
        model.fit(df, ct, 50)
        t2 = time.time()
        elapsed_time = t2-t1
        print(str(round(elapsed_time, 2)) + " seconds")
        
        model.save(data_set +'_scCMGAN'+str(persent)+'_' + ct + '_'+str(epoch)+'.pkl')

number of 'cell_1': 5
epoch 50 
2.3 seconds
number of 'cell_1': 5
epoch 100 
1.84 seconds
number of 'cell_1': 5
epoch 150 
1.83 seconds
number of 'cell_1': 5
epoch 200 
1.82 seconds
number of 'cell_1': 5
epoch 250 
1.82 seconds
number of 'cell_1': 5
epoch 300 
1.89 seconds
number of 'cell_2': 5
epoch 50 
2.84 seconds
number of 'cell_2': 5
epoch 100 
2.42 seconds
number of 'cell_2': 5
epoch 150 
1.8 seconds
number of 'cell_2': 5
epoch 200 
2.0 seconds
number of 'cell_2': 5
epoch 250 
1.89 seconds
number of 'cell_2': 5
epoch 300 
1.82 seconds
number of 'cell_3': 5
epoch 50 
1.97 seconds
number of 'cell_3': 5
epoch 100 
2.98 seconds
number of 'cell_3': 5
epoch 150 
2.39 seconds
number of 'cell_3': 5
epoch 200 
1.78 seconds
number of 'cell_3': 5
epoch 250 
1.8 seconds
number of 'cell_3': 5
epoch 300 
1.87 seconds


In [14]:
# num_data: [num_data_start, num_data_end+1, num_data_term]
num_data_start = 50
num_data_end = 500
num_data_term = 50

for i in range(len(cellType)):
    ct =  cellType[i]
    df_celtype = df[df["cellType"]==ct]

    for epoch in range(epoch_start, epoch_end+1, epoch_term):
        model = scCMGAN.load(data_set +'_scCMGAN'+str(persent)+'_' + ct + '_'+str(epoch)+'.pkl')

        for num_data in range(num_data_start, num_data_end+1, num_data_term):
                    t1 = time.time() 
                    synthetic_data = model.sample(num_data)
                    t2 = time.time()
                    elapsed_time = t2-t1
                    print(ct +"_"+ str(epoch) +"_"+str(num_data) + ":" + str(elapsed_time))
                    synthetic_data.to_csv("./synthetic_data_"+data_set+"_scCMGAN"+str(persent)+"_"+ ct +"_"+str(epoch)+"_" +  str(num_data) + ".csv", index= False)

cell_1_50_50:0.04510831832885742
cell_1_50_100:0.08382058143615723
cell_1_50_150:0.12842607498168945
cell_1_50_200:0.1651918888092041
cell_1_50_250:0.18824219703674316
cell_1_50_300:0.21574163436889648
cell_1_50_350:0.2701113224029541
cell_1_50_400:0.2952241897583008
cell_1_50_450:0.3054087162017822
cell_1_50_500:0.354738712310791
cell_1_100_50:0.043364763259887695
cell_1_100_100:0.08891916275024414
cell_1_100_150:0.13560914993286133
cell_1_100_200:0.14188146591186523
cell_1_100_250:0.2071523666381836
cell_1_100_300:0.2236480712890625
cell_1_100_350:0.2866365909576416
cell_1_100_400:0.30826663970947266
cell_1_100_450:0.4094064235687256
cell_1_100_500:0.47246217727661133
cell_1_150_50:0.06793355941772461
cell_1_150_100:0.10606503486633301
cell_1_150_150:0.15637731552124023
cell_1_150_200:0.20393037796020508
cell_1_150_250:0.275768518447876
cell_1_150_300:0.28975987434387207
cell_1_150_350:0.3492467403411865
cell_1_150_400:0.42966532707214355
cell_1_150_450:0.4621543884277344
cell_1_150_

## Integrate synthetic_data and original data

In [15]:
# persentage of cell marker genes (N =2)
persentage_1 = 40
persentage_2 = 20

# epoch: [epoch_start, epoch_end+1, epoch_term]
epoch_start = 50
epoch_end = 300
epoch_term = 50

# num_data: [num_data_start, num_data_end+1, num_data_term]
num_data_start = 50
num_data_end = 500
num_data_term = 50

# new sample name
sample_name  ="indiv3"

In [16]:
import pandas as pd
def train2train_aug(train_data_path, pheno_data_path, num_data, epoch, method, data_set):
    df = pd.read_csv(train_data_path, header=None)
    df.iloc[0, 0] = "cellType"
    df = df.T
    df.columns = df.iloc[0]
    df = df.drop(df.index[[0]])

    for ct in cellType:
        synthetic_data_ct = pd.read_csv("./synthetic_data_"+data_set+"_scCMGAN"+"20_"+ ct +"_"+str(epoch)+"_" +  str(num_data) + ".csv")
        synthetic_data_ct_2 = pd.read_csv("./synthetic_data_"+data_set+"_scCMGAN"+"40_"+ ct +"_"+str(epoch)+"_" +  str(num_data) + ".csv")
        if ct ==  cellType[0]:
            synthetic_data = pd.concat([synthetic_data_ct, synthetic_data_ct_2])
        else:    
            synthetic_data = pd.concat([synthetic_data, synthetic_data_ct, synthetic_data_ct_2])

    df_concat_multi = pd.concat([df, synthetic_data])
    df_concat_multi_T = df_concat_multi.T
    df_concat_multi_T[1:] = df_concat_multi_T[1:].round()
    df_concat_multi_T[1:] = df_concat_multi_T[1:].astype(int)

    df = pd.read_csv(train_data_path, header=None)
    gene_pd = df[0]
    df_concat_multi_T.set_axis(gene_pd, inplace=True)

    sh=df_concat_multi_T.shape
    df_concat_multi_T.columns=range(sh[1])
    df_concat_multi_T.columns = df_concat_multi_T.iloc[0]
    df_concat_multi_T = df_concat_multi_T.drop(df_concat_multi_T.index[[0]])

    df_concat_multi_T = df_concat_multi_T.clip(lower=0)

    # save
    df_concat_multi_T.to_csv('./'+method+'/'+data_set+ '_train_'+ method + "_" +str(epoch)+ "_" + str(num_data*2) + '.csv', header=True, index=True)


    # pheno data
    df_p = pd.read_csv(pheno_data_path)
    indiv = [sample_name]
    indiv *= df_concat_multi_T.shape[1] - df.shape[1]+1
    df_p_new = pd.DataFrame(
        data={'cellID': list(range(1,num_data*2*len(cellType)+1)), 
            'cellType': synthetic_data["cellType"],
            'sampleID': indiv}
    )


    df_p_concat = pd.concat([df_p, df_p_new])
    df_p_concat[""] = list(range(1,df_p_concat.shape[0]+1))
    df_p_concat = df_p_concat.reindex(columns=['', 'cellID', 'cellType','sampleID'])

    # save
    df_p_concat.to_csv('./'+method+'/'+data_set+ '_pDataC_'+ method + "_"+ str(epoch)+ "_" + str(num_data*2) + '.csv', header=True, index=False)

In [17]:
method = "scCMGAN"
num_datas = list(range(num_data_start, num_data_end+1, num_data_term)) 
epochs = list(range(epoch_start, epoch_end+1, epoch_term))

if not os.path.exists(method):
    os.makedirs(method)

for num_data in num_datas:
    for epoch in epochs:
            print("num_data: " + str(num_data)+", epoch: " + str(epoch))
            train2train_aug('./' + data_set + '_train.csv','./' + data_set + '_pDataC_train.csv', num_data, epoch, method, data_set)

num_data: 50, epoch: 50
num_data: 50, epoch: 100
num_data: 50, epoch: 150
num_data: 50, epoch: 200
num_data: 50, epoch: 250
num_data: 50, epoch: 300
num_data: 100, epoch: 50
num_data: 100, epoch: 100
num_data: 100, epoch: 150
num_data: 100, epoch: 200
num_data: 100, epoch: 250
num_data: 100, epoch: 300
num_data: 150, epoch: 50
num_data: 150, epoch: 100
num_data: 150, epoch: 150
num_data: 150, epoch: 200
num_data: 150, epoch: 250
num_data: 150, epoch: 300
num_data: 200, epoch: 50
num_data: 200, epoch: 100
num_data: 200, epoch: 150
num_data: 200, epoch: 200
num_data: 200, epoch: 250
num_data: 200, epoch: 300
num_data: 250, epoch: 50
num_data: 250, epoch: 100
num_data: 250, epoch: 150
num_data: 250, epoch: 200
num_data: 250, epoch: 250
num_data: 250, epoch: 300
num_data: 300, epoch: 50
num_data: 300, epoch: 100
num_data: 300, epoch: 150
num_data: 300, epoch: 200
num_data: 300, epoch: 250
num_data: 300, epoch: 300
num_data: 350, epoch: 50
num_data: 350, epoch: 100
num_data: 350, epoch: 150